### Préparation des données 

#### Import des librairies & Acquisition des données 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=Warning)
from sklearn.impute import KNNImputer
from sklearn.impute import MissingIndicator

In [2]:
data = pd.read_csv("data/06e9c61d-e45f-4d74-beb5-e5e53ac6d2de.csv")

In [3]:
data.head()

,Unnamed: 0,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,2072,-119.84,36.77,6.0,1853.0,473.0,1397.0,417.0,1.4817,72000.0,INLAND
1,10600,-117.80,33.68,8.0,2032.0,349.0,862.0,340.0,6.9133,274100.0,<1H OCEAN
2,2494,-120.19,36.60,25.0,875.0,214.0,931.0,214.0,1.5536,58300.0,INLAND
3,4284,-118.32,34.10,31.0,622.0,229.0,597.0,227.0,1.5284,200000.0,<1H OCEAN
4,16541,-121.23,37.79,21.0,1922.0,373.0,1130.0,372.0,4.0815,117900.0,INLAND


#### Changement des nom des colonnes

In [4]:
data = data.rename(columns={'Unnamed: 0': 'ID logement', 
                          'housing_median_age': 'Age moyen logement', 
                          'total_rooms': 'Total pieces', 
                          'total_bedrooms': 'Total chambres', 
                          'population': 'Nb personne logement', 
                          'households': 'Nb famille logement', 
                          'median_income': 'Salaire median logement',
                          'median_house_value': 'Valeur moyenne logement',
                          'ocean_proximity': 'Proximation Ocean'})

In [5]:
data.head()

,ID logement,longitude,latitude,Age moyen logement,Total pieces,Total chambres,Nb personne logement,Nb famille logement,Salaire median logement,Valeur moyenne logement,Proximation Ocean
0,2072,-119.84,36.77,6.0,1853.0,473.0,1397.0,417.0,1.4817,72000.0,INLAND
1,10600,-117.80,33.68,8.0,2032.0,349.0,862.0,340.0,6.9133,274100.0,<1H OCEAN
2,2494,-120.19,36.60,25.0,875.0,214.0,931.0,214.0,1.5536,58300.0,INLAND
3,4284,-118.32,34.10,31.0,622.0,229.0,597.0,227.0,1.5284,200000.0,<1H OCEAN
4,16541,-121.23,37.79,21.0,1922.0,373.0,1130.0,372.0,4.0815,117900.0,INLAND


#### Vérification de la présence de doublons 

In [6]:
def doublon_column(column):
    new_column = column.drop_duplicates()
    if not len(column) == len(new_column):
        return True
    return False

In [7]:
doublon_column(data['ID logement'])

False

#### Imputation des données manquantes

In [8]:
def replace_missing_value(column):
    imputer = KNNImputer(n_neighbors=1)
    new_column = imputer.fit_transform(column)
    return new_column

In [9]:
total_chambres = replace_missing_value(data[["Total chambres"]])
data["Total chambres"] = total_chambres

#### Encodage des variables catégorielles

In [10]:
data["Proximation Ocean"].value_counts()

<1H OCEAN     7312
INLAND        5235
NEAR OCEAN    2140
NEAR BAY      1821
ISLAND           4
Name: Proximation Ocean, dtype: int64

In [11]:
data = pd.get_dummies(data, columns=['Proximation Ocean'], prefix=[""])
data.head()

,ID logement,longitude,latitude,Age moyen logement,Total pieces,Total chambres,Nb personne logement,Nb famille logement,Salaire median logement,Valeur moyenne logement,_<1H OCEAN,_INLAND,_ISLAND,_NEAR BAY,_NEAR OCEAN
0,2072,-119.84,36.77,6.0,1853.0,473.0,1397.0,417.0,1.4817,72000.0,0,1,0,0,0
1,10600,-117.80,33.68,8.0,2032.0,349.0,862.0,340.0,6.9133,274100.0,1,0,0,0,0
2,2494,-120.19,36.60,25.0,875.0,214.0,931.0,214.0,1.5536,58300.0,0,1,0,0,0
3,4284,-118.32,34.10,31.0,622.0,229.0,597.0,227.0,1.5284,200000.0,1,0,0,0,0
4,16541,-121.23,37.79,21.0,1922.0,373.0,1130.0,372.0,4.0815,117900.0,0,1,0,0,0


### Baseline  : Modèle de base

#### Import du modèle

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#### Séléction des Features X

In [13]:
X = data[["ID logement","longitude","latitude","Age moyen logement",
          "Total pieces","Total chambres","Nb personne logement",
          "Nb famille logement","Salaire median logement","_<1H OCEAN",
          "_INLAND","_ISLAND","_NEAR BAY","_NEAR OCEAN"
         ]]
X = np.array(X)

#### Séléction de la Target Y

In [14]:
Y = data[["Valeur moyenne logement"]]
Y = np.array(Y)

#### On divise notre data set pour avoir les données d'entrainement et d'évaluation.

In [15]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.20)

#### Entrainement & évaluation du modèle

In [18]:
model = LinearRegression()
model.fit(x_train, y_train)
print(f"Notre baseline est de : {model.score(x_test, y_test)}")

Notre baseline est de : 0.6667937706027622
